In [ ]:
import pandas as pd

# Load in the AVL data

### Format DateTime columns, Remove NAN's

In [4]:
# I have a theoretical process for doing this programatically, where
# I don't need to assume the column names.

headers = ['REV', 'REPORT_TIME', 'VEHICLE_TAG', 'LONGITUDE', 'LATITUDE',
           'SPEED', 'HEADING', 'TRAIN_ASSIGNMENT', 'PREDICTABLE']

avl_df = pd.read_csv('data/avl/sfmtaAVLRawData01032016.csv', skiprows=[0], names=headers,
                dtype={2: str, 7: str}, parse_dates=[1])

avl_df = avl_df.dropna(axis=0)

### Get the date of the data from the first row

In [5]:
data_date = avl_df.iloc[0]['REPORT_TIME']

### Load in the block and sign reference data

In [26]:
blockref_df = pd.read_csv('data/lookUpBlockIDToBlockNumNam.csv')
signref_df = pd.read_csv('data/lookUpSignUpPeriods.csv', parse_dates=[2,3])

### Get the SignID during which our data is valid

In [27]:
sign_id = signref_df[(signref_df['FROMDATE'] < data_date) & (signref_df['TODATE'] > data_date)]['SIGNID'].values[0]

### Get the block which occur during the SignID

In [28]:
date_blocks = blockref_df[blockref_df['SIGNID'] == sign_id]

In [39]:
date_blocks.head()

,SIGNID,BLOCKID,BLOCKNUM,BLOCKNAME
11,100,511037,6009,6009
12,100,517357,9302,9302
13,100,517361,9306,9306
14,100,532326,9401,9401
15,100,532330,9409,9409


# Load in the GTFS data

### Load in the Routes

In [32]:
routes = pd.read_csv('data/gtfs/sfmta_2015-11-01/routes.txt')

### Load in the Trips

In [33]:
trips = pd.read_csv('data/gtfs/sfmta_2015-11-01/trips.txt')

### Get the Route ID for the 33 Bus

In [34]:
route_33_id = routes[routes['route_short_name'] == '33']['route_id'].values[0]

### Get all 33 trips, going in the same direction

In [36]:
trips_33_dir0 = trips[(trips['route_id'] == route_33_id) & (trips['direction_id'] == 0)]

### Get all the 33 trip block_id's

In [38]:
trip33_blocks = trips_33_dir0['block_id'].unique()

# Match trip block_id's to the Date-constrained BlockRef Blocks

In [40]:
dateblock_33 = date_blocks[date_blocks['BLOCKNUM'].isin(trip33_blocks)]

### Get all the BlockNames

In [46]:
dateblock33_names = dateblock_33['BLOCKNAME'].values

# Filter AVL Data based on BlockNames

In [47]:
avl_33dir_df = avl_df[avl_df['TRAIN_ASSIGNMENT'].isin(dateblock33_names)]

In [50]:
avl_33dir_df.head()

,REV,REPORT_TIME,VEHICLE_TAG,LONGITUDE,LATITUDE,SPEED,HEADING,TRAIN_ASSIGNMENT,PREDICTABLE
157426,1506,2016-01-03 09:47:39,5412,-122.43882,37.76066,7.500,267.0,3305,1
157427,1506,2016-01-03 09:50:15,5412,-122.44467,37.75970,3.889,243.0,3305,1
157428,1506,2016-01-03 09:53:15,5412,-122.44671,37.76167,0.000,355.0,3305,1
157429,1506,2016-01-03 09:53:36,5412,-122.44693,37.76227,6.667,351.0,3305,1
157430,1506,2016-01-03 09:55:21,5412,-122.44644,37.76736,2.500,347.0,3305,1
